# Preparation

## Read data

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
input_file_path = 'data/life_data.csv'
output_interval_file_path = 'data/life_interval_data.csv'
output_total_file_path = 'data/life_total_data.csv'

In [5]:
df = pd.read_csv(input_file_path,header=None,parse_dates=[[0,1]],infer_datetime_format=True)
df.columns = ['time','activity']
df.head()

,time,activity
0,2019-05-01 05:20:00,prep
1,2019-05-01 05:35:00,math
2,2019-05-01 06:35:00,pause
3,2019-05-01 06:45:00,math
4,2019-05-01 07:45:00,music


## Fix dates

There is a problem with the data: A time such as `2019-04-11 00:35` often refers to the day after (since it is after midnight)! This can be corrected by finding all dates that violate the chronological order and adding one day (see algorithm below).

Before changing the data, a new column is introduced that keeps the original date. This way, no information is lost. It will later be useful to assign sleep duration to dates.

In [6]:
df['assigned_to'] = df.time.copy().map(lambda x: x.replace(hour=0,minute=0))
df.head()

,time,activity,assigned_to
0,2019-05-01 05:20:00,prep,2019-05-01
1,2019-05-01 05:35:00,math,2019-05-01
2,2019-05-01 06:35:00,pause,2019-05-01
3,2019-05-01 06:45:00,math,2019-05-01
4,2019-05-01 07:45:00,music,2019-05-01


### Algorithm

The algorithm below uses an inplace approach. It works with more than one faulty datetime -- such as having both `01:00,work` and `03:00,sleep` in one file. The reason is that adding one day to a faulty date creates another violation of chronological order that is caught in the next run of the loop. The only constraint is that the latest time of a day has to appear before the earliest of the next.

In [7]:
for i in range(len(df)):
    if i != 0 and df.iloc[i].time < df.iloc[i-1].time:
        print('Changed:', i)
        df.loc[i,'time'] += dt.timedelta(days=1)
df.head()

Changed: 38
Changed: 39
Changed: 40


,time,activity,assigned_to
0,2019-05-01 05:20:00,prep,2019-05-01
1,2019-05-01 05:35:00,math,2019-05-01
2,2019-05-01 06:35:00,pause,2019-05-01
3,2019-05-01 06:45:00,math,2019-05-01
4,2019-05-01 07:45:00,music,2019-05-01


## df_i: Convert to intervals

Convert the absolute times to intervals by calculating the differences. For instance, the two first rows...

In [8]:
df.drop('assigned_to',axis=1).head(2)

,time,activity
0,2019-05-01 05:20:00,prep
1,2019-05-01 05:35:00,math


... should be converted to: `2019-05-01|15|prep`

In [9]:
df_i = pd.DataFrame(columns=['date','time','activity'])

for (i,row) in df.iterrows():
    if i != 0:
        add_row = pd.Series({
            'date':df.iloc[i-1].assigned_to,
            'time':row.time-df.iloc[i-1].time,
            'activity':df.iloc[i-1].activity,
            'start_time':df.iloc[i-1].time
        })
        df_i = df_i.append(add_row,ignore_index=True)

# Sanity check
print(len(df)==len(df_i)-1)

print(df_i.dtypes)
df_i.head()

False
date           datetime64[ns]
time          timedelta64[ns]
activity               object
start_time     datetime64[ns]
dtype: object


,date,time,activity,start_time
0,2019-05-01,00:15:00,prep,2019-05-01 05:20:00
1,2019-05-01,01:00:00,math,2019-05-01 05:35:00
2,2019-05-01,00:10:00,pause,2019-05-01 06:35:00
3,2019-05-01,01:00:00,math,2019-05-01 06:45:00
4,2019-05-01,01:30:00,music,2019-05-01 07:45:00


## df_t: Sum up intervals

Another useful view on the data is to store the total hours that were spent doing an activity, for every day and every activity.

In [10]:
unique_activities = df_i.activity.unique()

def by_activity(df):
    res = df.groupby('activity').sum()
    return pd.DataFrame([res.time.values],columns=res.index.values)

df_t = df_i.groupby('date').apply(by_activity)
df_t = df_t.fillna(dt.timedelta(0))
df_t = df_t.droplevel(1)
df_t.head()

,cook,eat,math,music,pause,prep,sleep,uni,meditation,special,work
date,,,,,,,,,,,
2019-05-01,00:33:00,01:46:00,03:31:00,01:57:00,01:13:00,05:49:00,11:43:00,01:38:00,00:00:00,0 days,0 days
2019-05-02,00:41:00,02:27:00,06:39:00,00:57:00,00:31:00,01:45:00,05:55:00,02:50:00,00:00:00,0 days,0 days
2019-05-03,00:56:00,01:57:00,00:00:00,00:10:00,00:52:00,03:50:00,12:15:00,05:45:00,00:15:00,0 days,0 days
2019-05-04,01:20:00,02:04:00,02:30:00,00:00:00,00:00:00,01:39:00,06:52:00,06:27:00,00:15:00,0 days,0 days
2019-05-05,00:17:00,02:18:00,02:45:00,01:00:00,00:00:00,05:01:00,08:26:00,05:17:00,00:20:00,0 days,0 days


## Write to files

In [12]:
def format_timedelta(td):
    # Hack for easy formatting
    return (td + pd.to_datetime('2000-01-01')).strftime('%H:%M')
    
print(format_timedelta(pd.to_timedelta('2h 30min')))

02:30


In [13]:
df_i_tmp = df_i.copy()
df_i_tmp.time = df_i_tmp.time.map(format_timedelta)
df_i_tmp.date = df_i_tmp.date.map(lambda x: x.strftime('%Y-%m-%d'))

df_i_tmp.to_csv(output_interval_file_path,index=None)

In [14]:
df_t.applymap(format_timedelta).to_csv(output_total_file_path)